In [1]:
import torch.nn
import torch.nn.functional as F
import torch
import pickle
import torchvision
from tqdm import tqdm_notebook
import numpy as np
from skimage import io
from pathlib import Path
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from matplotlib import colors, pyplot as plt
import matplotlib.pyplot as plt
import pandas as pd
from torch.autograd import Variable
import torchvision.transforms as transforms
import PIL.Image as Image
import PIL
import time
import cv2
from scipy.interpolate import make_interp_spline, BSpline, interp1d
import operator
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
import os
from torch.utils.data import DataLoader, RandomSampler, Dataset
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

ToPIL = transforms.ToPILImage()

In [2]:
torch.cuda.is_available()

True

In [3]:
class ShipsDataset(Dataset):
    def __init__(self, files, mode, transform, csv_file = None):
        assert mode in ['train', 'test', 'val']
        
        self.files = files
        self.mode = mode
        self.transform = transform
        self.len_ = len(self.files)
        self.filters = [None]
        if (mode != 'test'):
            self.labels = [csv_file.loc[str(file).split('/')[-1]]['IdCls'] if type(csv_file.loc[str(file).split('/')[-1]]['IdCls']) == np.int64 else 1 for file in files]
            self.label_encoder = LabelEncoder()
            self.label_encoder.fit(self.labels)
        
    def __len__(self):
        return self.len_
    
    def load_img(self, fname):
        fname = str(fname)
        img = Image.open(fname)
        img = np.array(Image.open(fname))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.blur(img, (11,11))
        img = cv2.Laplacian(img, cv2.THRESH_BINARY, scale=0.55, ksize=5)
        img = Image.fromarray(np.uint8(img))
        return img
    
    def __getitem__(self, idx):
        x = self.load_img(fname=self.files[idx])
        x = self.transform(x)
        
        if self.mode == 'test':
            return x
        else:
            y = self.labels[idx]
            y = self.label_encoder.transform([y]).item()
            return x, y

In [4]:
def image_dims(source):
    """
    Получение размеров входных изображений
    :param source (DataLoader, ndarray, Tensor, Jpeg): Данные для извлечения размера изображения
    :return: Размеры изображения (tuple)
    """
    if type(source) is torch.utils.data.dataloader.DataLoader:
        image, _ = iter(source).next()
        return (image.shape[-2], image.shape[-1])
    if type(source) is PIL.JpegImagePlugin.JpegImageFile:
        source = np.array(source)
    if type(source) is np.ndarray or type(source) is torch.Tensor:
        shapes = source.shape[-3:]
        if shapes[2] == 3 or shapes[2] == 1:
            return (shapes[0], shapes[1])
        else:
            return (shapes[1], shapes[2])
    raise Exception('Unknown type of file')


Данные функции нужны для работы с файлами 'Losses.txt' и 'CNNs.txt'. Они имеют строгую структуру и нужны для сравнения различных нейросетей

In [5]:
def get_loss(path, n):
    """
    Преобразование файла с losses в переменную (для построения графиков)
    :param path (string): Путь к файлу, из которого извлекаются данные
    :param n (string, int): Номер нейронной сети для извлечения
    :return: Losses данной нейронной сети (list)
    """
    with open(path, 'r') as file:
        t = list(filter(bool, file.read().split('&')))
        for i in t:
            if int(i.split('\n')[0]) == int(n):
                t = i.split('\n')[1]
                break
        try:
            t = t.split()
        except:
            raise Exception('Ошибка в losses {}. Невозможно получить данные'.format(n))
        t[0] = t[0][1:]
        t.pop(-1)
        return list(map(lambda x: np.round(float(x), 3), t))


def get_all_losses(path):
    """
    "Достаёт" все losses из файла
    :param path (string): Путь к файлу для извлечения losses
    :return: Все losses из файла (dict)
    """
    with open(path, 'r') as file:
        keys = []
        for i in list(filter(bool, file.read().split('&'))):
            keys.append(int(i.split('\n')[0]))
        losses = [get_loss(path, i) for i in keys]
        return {keys[i]: losses[i] for i in range(len(keys))}


def get_losses(path, keys):
    """
    "Достаёт" losses указанных сетей из файла
    :param path (string): Путь к файлу
    :param keys (list, tuple, ndarray): Номера сетей losses которых надо извлечь
    :return: Losses указанных сетей (dict)
    """
    with open(path, 'r') as file:
        losses = [get_loss(path, i) for i in sorted(keys)]
        return {sorted(keys)[i]: losses[i] for i in range(len(keys))}



In [6]:
def check_size_losses(path, size=124, logging=True):
    """
    Проверка размеров losses в указанном файле
    :param path (string): Путь к файлу
    :param size (int): Предпологаемый размер каждого лосса
    :param logging (bool): Флаг вывода изменений в файле
    :return: Losses, соответствующие предпологаемому размеру (dict). Флаг наличия изменений в файле
    """
    losses = get_all_losses(path)
    tmp = []
    changed = False
    for i in losses.keys():
        if len(losses[i]) < size:
            if logging:
                print('Длина losses сети {} меньше обозначенной. Losses данной сети стёрты'.format(i))
            changed = True
            continue
        if len(losses[i]) > size:
            if logging:
                print('Длина losses сети {} больше обозначенной. Losses сокращены до {}'.format(i, size))
            changed = True
            tmp.append([i, losses[i][:size]])
            continue
        tmp.append([i, losses[i]])
    return {tmp[i][0]: tmp[i][1] for i in range(len(tmp))}, changed


def standard_losses(path='Losses.txt', size=124, logging=True):
    """
    Приводит файл с losses к стандартному виду (нумерация с единицы, размеры приведены к size)
    :param path (string): Путь к файлу
    :param size (int): Предпологаемый размер всех losses
    :return: None
    """
    old_losses = get_all_losses(path)
    losses, changed = check_size_losses(path, size, logging)
    tmp = ''
    if changed:
        with open(path, 'r') as file:
            tmp = file.read()
        with open('Old_{}'.format(path), 'w') as file:
            file.write(tmp)

        tmp = ''
        print('\n')
        for i, j in enumerate(losses.keys()):
            tmp += '&' + str(i + 1) + '\n['
            for num in losses[j]:
                tmp += str(num) + ' '
            tmp += ']\n'
            if j != i + 1:
                print('{} --> {}'.format(j, i + 1))
        with open(path, 'w') as file:
            file.write(tmp)

        print('\n\nФайл {} перезаписан\nСтарая версия сохранена в файл Old_{}'.format(path, path))
    else:
        for i, j in enumerate(losses.keys()):
            tmp += '&' + str(i + 1) + '\n['
            for num in losses[j]:
                tmp += str(num) + ' '
            tmp += ']\n'
            if j != i + 1:
                print('{} --> {}'.format(j, i + 1))
        with open(path, 'w') as file:
            file.write(tmp)
        print('\n\nФайл {} перезаписан'.format(path))


def merge_losses(path1, path2, size=124):
    """
    Объединият два файла с losses в один файл стандартного вида
    :param path1 (string): Путь к первому файлу
    :param path2 (string): Путь ко второму файлу
    :param size (int): Предпологаемый размер всех losses
    :return: None
    """
    if check_size_losses(path1, size, False)[1]:
        print('Файл {} не приведён к стандартному виду. Провести стандартизацию?'.format(path1))
        if str(input()) != '0':
            print('\n{}:\n'.format(path1))
            standard_losses(path1, size)
            print('-----------')
        else:
            raise Exception('Невозможно объединить файлы нестандартного вида')

    if check_size_losses(path2, size, False)[1]:
        print('Файл {} не приведён к стандартному виду. Провести стандартизацию?'.format(path2))
        if str(input()) != '0':
            print('\n{}:\n'.format(path2))
            standard_losses(path2, size)
            print('-----------')
        else:
            raise Exception('Невозможно объединить файлы нестандартного вида')

    losses1 = get_all_losses(path1)
    losses2 = get_all_losses(path2)
    print('\nMerging:\nНомера losses в фале {} не изменились\nНомера losses в файле {} перешли в следующие:\n'.format(
        path1, path2))
    for key, loss in losses2.items():
        if loss not in losses1.values():
            losses1[list(losses1.keys())[-1] + 1] = loss
            print('{} --> {}'.format(key, list(losses1.keys())[-1]))
    with open('Losses.txt', 'w') as file:
        for i, loss in enumerate(losses1.values()):
            file.write('&' + str(i + 1) + '\n[')
            for l in loss:
                file.write(str(l) + ' ')
            file.write(']\n')



In [7]:
def check_standard_cnns(path):
    """
    Проверка файла с нейросетями на стандартный вид
    :param path (string): Путь к файлу
    :return: Флаг приведённости к стандартному виду
    """
    tmp = ''
    with open(path, 'r') as file:
        tmp = list(filter(bool, file.read().split('--')))
        for i in range(len(tmp)):
            tmp[i] = tmp[i].split('ConvNetwork')
    keys = [int(''.join(list(filter(lambda x: 48 <= ord(x) <= 58, tmp[i][0])))) for i in range(len(tmp))]
    if keys == list(range(1, len(tmp) + 1)):
        return True
    else:
        return False


def standard_cnns(path='CNNs.txt'):
    """
    Приводит файл с нейросетями к стандартному виду (нумерация с единицы)
    :param path (string): Путь к файлу
    :return: None
    """
    tmp = []
    tmp1 = ''
    with open(path, 'r') as file:
        tmp1 = file.read()
    with open(path, 'r') as file:
        tmp = list(filter(bool, file.read().split('--')))
        for i in range(len(tmp)):
            tmp[i] = tmp[i].split('ConvNetwork')
    with open('Old_{}'.format(path), 'w') as file:
        file.write(tmp1)

    with open(path, 'w') as file:
        for i in range(len(tmp)):
            num = int(''.join(list(filter(lambda x: 48 <= ord(x) <= 58, tmp[i][0]))))
            if num != i + 1:
                print('{} --> {}'.format(num, i + 1))
            file.write('-----------------------\n' + str(i + 1) + '\nConvNetwork' + tmp[i][1])
    print('Файл {} перезаписан. Старая версия сохранена в файл Old_{}'.format(path, path))


def merge_cnns(path1, path2):
    """
    Объединяет два файла с нейросетями в один файл стандартного вида
    :param path1 (string): Путь к первому файлу
    :param path2 (string): Путь ко второму файлу
    :return: None
    """
    tmp1 = []
    tmp2 = []
    if not check_standard_cnns(path1):
        print('Файл {} не приведён к стандартному виду. Провести стандартизацию?'.format(path1))
        if str(input()) != '0':
            print('\n{}:\n'.format(path1))
            standard_cnns(path1)
            print('-----------')
        else:
            raise Exception('Невозможно объединить файлы нестандартного вида')

    if not check_standard_cnns(path2):
        print('Файл {} не приведён к стандартному виду. Провести стандартизацию?'.format(path2))
        if str(input()) != '0':
            print('\n{}:\n'.format(path2))
            standard_cnns(path2)
            print('-----------')
        else:
            raise Exception('Невозможно объединить файлы нестандартного вида')

    with open(path1, 'r') as file:
        tmp1 = list(filter(bool, file.read().split('--')))
        for i in range(len(tmp1)):
            tmp1[i] = tmp1[i].split('ConvNetwork')
            tmp1[i][0] = int(''.join(list(filter(lambda x: 48 <= ord(x) <= 58, tmp1[i][0]))))
    with open(path2, 'r') as file:
        tmp2 = list(filter(bool, file.read().split('--')))
        for i in range(len(tmp2)):
            tmp2[i] = tmp2[i].split('ConvNetwork')
            tmp2[i][0] = int(''.join(list(filter(lambda x: 48 <= ord(x) <= 58, tmp2[i][0]))))
    cnns = [tmp1[i][1] for i in range(len(tmp1))]
    print(
        '\nMerging:\nНомера сетей в фале {} не изменились\nНомера сетей в файле {} перешли в следующие:\n'.format(path1,
                                                                                                                  path2))
    for i, cnn in tmp2:
        if cnn not in cnns:
            print('{} --> {}'.format(i, len(tmp1) + 1))
            tmp1.append([len(tmp1) + 1, cnn])
            cnns.append(cnn)
    with open('CNNs.txt', 'w') as file:
        for i in range(len(tmp1)):
            file.write('-----------------------\n' + str(tmp1[i][0]) + '\nConvNetwork' + tmp1[i][1])


def next_cnn(path):
    """
    Вычисляет номер следующей нейросети
    :param path (string): Путь к файлу с нейросетями
    :return: Номер следующей нейросети
    """
    try:
        open(path, 'r')
    except:
        with open(path, 'w') as file:
            file.write('')
        return 1
    if not check_standard_cnns(path):
        print('Файл {} не приведён к стандартному виду. Провести стандартизацию?'.format(path))
        if str(input()) != '0':
            print('\n{}:\n'.format(path))
            standard_cnns(path)
            print('-----------')
    with open(path, 'r') as file:
        
        tmp = list(filter(bool, file.read().split('--')))
        return len(tmp) + 1


In [8]:
def maps_dims(im_dims, conv_size, pool_size, conv_stride, pool_stride):
    """
    Вычисление размеров карт признаков после свёрточных слоёв
    :param im_dims (tuple, list, ndarray): Размеры входного изображения
    :param conv_size (tuple, list, ndarray): Размер фильтра на каждом слое
    :param pool_size (tuple, list, ndarray): Размер ядра пулинга на каждом слое
    :param conv_stride (tuple, list, ndarray): Величина шага фильтра свёртки на каждом слое
    :param pool_stride (tuple, list, ndarray): Величина шага ядра пулинга на каждом слое
    :return: Размеры карт признаков после прохождения свёрточных слоёв
    """
    h, w = im_dims
    for i in range(len(conv_size)):
        h = (h - conv_size[i]) // conv_stride[i] + 1
        w = (w - conv_size[i]) // conv_stride[i] + 1
        if pool_size[i]:
            h = (h - pool_size[i]) // pool_stride[i] + 1
            w = (w - pool_size[i]) // pool_stride[i] + 1
    return (h, w)



In [9]:
def get_added_picture(image, filter):
    """
    Применяет к изображению фильтр (нужно для аугментации)
    :param image: Изображение, которое нужно изменить
    :param filter: Фильтр, который нужно применить
    :return: Изменённое изображение
    """
    return image.filter(filter) if filter else image


def pic_num(path):
    """
    Функция вычисляет номер изображения в файле типа "<путь>/pic_<номер>.jpg"
    :param path: Путь к изображению
    :return: Номер изображения
    """
    return int(str(path).split('pic_')[1][:4])


Чтобы не перегружать класс нейросети ненужными строками кода, все слои разобьём на несколько составляющих и будем держать их в списках, объединённых в один большой словарь. Его мы передаём в класс нейросети, в котором все эти составляющие будут поочерёдно применяться ко входным данным.

Незначащие параметры вынесем в функции, генерирующие списки слоёв этих параметров

In [10]:
def conv_addition(input, hn, hl, conv_size, conv_stride):
    """
    Создание свёрточных слоёв по заданным параметрам
    :param input (int): Количество каналов во входном изображении
    :param hn (list, tuple, ndarray): Количество фильтров на каждом слое
    :param hl (int): Количество слоёв
    :param conv_size (list, tuple, ndarray): Размер фильтра на каждом слое
    :param conv_stride (list, tuple, ndarray): Величина шага фильтра свёртки на каждом слое
    :return: Свёрточные слои нейросети (list)
    """
    convolutions = \
        [
            torch.nn.Conv2d(in_channels=input, out_channels=hn[0],
                            kernel_size=conv_size[0],
                            stride=conv_stride[0])
        ]  # Объявление свёртки для первого слоя
    for i in range(hl - 1):  # Объяление свёрток для остальных слоёв
        convolutions.append(
            torch.nn.Conv2d(in_channels=hn[i], out_channels=hn[i + 1],
                            kernel_size=conv_size[i + 1], stride=conv_stride[i + 1]))
    return convolutions
        
    
def pool_addition(hl, pool_size, pool_stride):
    """
    Создание слоёв пулинга по заданным параметрам
    :param hl (int): Количество слоёв
    :param pool_size (list, tuple, ndarray): Размер ядра пулинга на каждом слое
    :param pool_stride (list, tuple, ndarray): Величина шага ядра пулинга на каждом слое
    :return: Пулинг-слои нейронной сети (list)
    """
    pool = []
    for i in range(hl): 
        if pool_size[i]:
            pool.append(torch.nn.MaxPool2d(kernel_size=pool_size[i], stride=pool_stride[i]))
        else:
            pool.append(None)
    return pool
    
    
def linears_addition(input, hn, hl, output):
    """
    Создание линейных слоёв по заданным параметрам
    :param input (int): Количество input-нейронов
    :param hn (list, tuple, ndarray): Количество нейронов на каждом слое
    :param hl (int): Количество полносвязных слоёв
    :param output (int): Количество output-нейронов
    :return: Линейные слои нейронной сети (list)
    """
    linears = []
    linears = [torch.nn.Linear(input, hn[0])]
    for i in range(hl - 1):
        linears.append(
            torch.nn.Linear(hn[i], hn[i + 1]))
    linears.append(torch.nn.Linear(hn[-1], output))
    return linears


In [11]:
def draw_plot(path, number, size=None, color='r'):
    """
    Отрисовывает график losses одной нейросети, применяя нелинейную регрессию
    :param path (string): Путь к файлу
    :param number (int, string): Номер нейросети для отрисовки
    :param size (int): Предпологаемый размер losses
    :param color (string): Цвет для отрисовки
    :return: None
    """
    if not size:
        size = len(get_loss(path, number))
    
    x = np.arange(size)
    y = np.array(get_loss(path, number))

    x = x[:, np.newaxis]
    y = y[:, np.newaxis]

    polynomial_features = PolynomialFeatures(degree=7)
    x_poly = polynomial_features.fit_transform(x)

    model = LinearRegression()
    model.fit(x_poly, y)
    y_poly_pred = model.predict(x_poly)

    rmse = np.sqrt(mean_squared_error(y, y_poly_pred))
    r2 = r2_score(y, y_poly_pred)

    sort_axis = operator.itemgetter(0)
    sorted_zip = sorted(zip(x, y_poly_pred), key=sort_axis)
    x, y_poly_pred = zip(*sorted_zip)
    plt.plot(x, y_poly_pred, color=color, label=str(number))


def show_losses(path, numbers, size, colors):
    """
    Отрисовывает графики losses указанных нейросетей
    :param numbers (list, tuple): Номера сетей для отрисовки
    :param size (int): Предпологаемый размер losses
    :param colors (list, tuple): Набор цветов для отрисовки
    :return: None
    """
    plt.figure(figsize=(15, 10))
    for i, j in enumerate(numbers):
        draw_plot(path, j, size, colors[i])
    plt.legend(numbers)


In [12]:
def train(train_params):
    """
    Обучает нейросеть на обучающей выборке
    :param train_params (dict): Все параметры для обучения нейросети
    :return: Обученная нейросеть
    """

    batch_size = train_params['primary_batch']
    trainloader = DataLoader(train_params['trainset'], batch_size=batch_size, shuffle=True)
    batch_decrease = train_params['batch_decrease']
    net = train_params['primary_net']
    num_epochs = train_params['num_epochs']
    write_seq = train_params['write_seq']
    show_seq = train_params['show_seq']
    lr = train_params['lr'] if train_params['lr'] else 0.001
    lr_decrease = train_params['lr_decrease'] if train_params['lr_decrease'] else 1
    loss_fn = train_params['loss_fn']
    
    net.train()

    if net.params['new'] and net.params['write']:
        f = open('Losses.txt', 'a')
        f.write('&{}'.format(net.params['name']) + '\n' + '[')
        f.close()
        net.params['new'] = False

    learning_rate = lr
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
    losses = []

    current_loss = 0.0
    accuracy = []

    for epoch in tqdm_notebook(range(num_epochs)):
        f = open('Losses.txt', 'a')

        running_loss = 0.0

        for i, batch in enumerate(tqdm_notebook(trainloader)):
            X_train, y_train = batch
            try:
                X_train, y_train = X_train.cuda(), y_train.cuda()
            except:
                pass
            optimizer.zero_grad()

            y_pred = net(X_train)
            loss = loss_fn(y_pred, y_train)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            current_loss += loss.item()

            if i % write_seq == write_seq - 1:
                losses.append(current_loss / write_seq)
                current_loss = 0.0

            if i % show_seq == show_seq - 1:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / show_seq))
                running_loss = 0.0

                if net.params['write']:
                    torch.save(net, 'CNN{}'.format(net.params['name']))

        learning_rate /= lr_decrease
        optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

        if epoch % 2:
            batch_size = int(batch_size // batch_decrease)
            batch_size = max(4, batch_size)
            trainloader = DataLoader(train_params['trainset'], batch_size=batch_size, shuffle=True)
            
        np.array(losses).round(3)
        if net.params['write']:
            for i in losses:
                f.write(str(i) + ' ')
            losses = []
            f.write(']\n' if epoch + 1 == num_epochs else ' ')
            f.close()

    print('Обучение закончено')
    return net


In [13]:
def validation(val_params):
    """
    Валидирует нейросеть на предложенном датасете
    :param val_params (dict): Все параметры для валидации 
    :return: None
    """
    net = val_params['net']
    batch_size = val_params['batch_size']
    valloader = DataLoader(val_params['valset'], batch_size=batch_size, shuffle=False)
    loss_fn = val_params['loss_fn']
    
    net.eval()
    correct = 0
    running_loss = 0.0
    total = valloader.__len__() * valloader.batch_size
    for X_val, y_val in tqdm_notebook(valloader):
        
        X_val, y_val = X_val.cuda(), y_val.cuda()
        
        y_pred = net(X_val)
 
        loss = loss_fn(y_pred, y_val).data
        _, predicted = torch.max(y_pred.data, 1)
 
        correct += (predicted == y_val).sum().item()
        running_loss += loss
 
    print("Validation Loss: {}".format(running_loss / len(valloader)))
    print(correct / total * 100, '%')
 
    if net.params['write']:
        with open('CNNs.txt', 'a') as file:
            file.write('--------------------------------\n' + str(net.params['name']) + '\n' + str(net) + '\n\n')
            file.write("Validation Loss: {}".format(running_loss / len(valloader)) + '\n')
            file.write("Total accuracy: %d %%" % (correct / total * 100) + '\n')

In [ ]:

def test(test_params):
    """
    :param test_params (dict): Все параметры для тестирования нейросети
    :return: None
    """
    testset = test_params['testset']
    batch_size = test_params['batch_size']
    net = test_params['net']
    net.eval()
    
    testloader = DataLoader(testset, batch_size=batch_size, shuffle=False)
    
    with open('submission.csv', 'w') as file:
        file.write('ImageId,IdCls\n')
        for i, X_test in enumerate(testloader):
            X_test = X_test.cuda()
            y_pred = net(X_test)
            _, predicted = torch.max(y_pred.data, 1)
            
            for j, pred in enumerate(predicted):
                file.write(str(testloader.dataset.files[i * batch_size + j]).split('/')[-1] + ',' + str(pred.item()) + '\n')
    print('Тестирование завершено, результаты сохранены')

In [15]:
def show_accuracy(valloader, net):
    y_val_all = torch.Tensor().long()
    predictions_all = torch.Tensor().long()

    # Пройдём по всему validation датасету и запишем ответы сети
    with torch.no_grad():
        for X, y in valloader:
            predictions = net(X)
            y_val = y
            _, predictions = torch.max(predictions, 1)

            # Аналог append для list
            y_val_all = torch.cat((y_val_all, y_val), 0)
            predictions_all = torch.cat((predictions_all, predictions), 0)

    class_correct = [0 for i in range(19)]
    class_total = [0 for i in range(19)]

    c = (predictions_all == y_val_all).squeeze()
    for i in range(len(predictions_all)):
        label = predictions_all[i]
        class_correct[label] += c[i].item()
        class_total[label] += 1

    print(class_total)

    for i in range(len(valloader.dataset.labels)):
        print('Accuracy of %5s : %2d %%' % (
            (dataset.labels[i], (100 * class_correct[i] / class_total[i]) if class_total[i] != 0 else -1)))

In [16]:

class ConvNetwork(torch.nn.Module):

    def __init__(self, p):
        """
        Инициализация нейронной сети
        :param p (dict): словарь параметров для нейронной сети
        """
        super(ConvNetwork, self).__init__()

        if len(p['fc']['drop_out']) != p['fc']['hl']:  # дополнение раздела 'drop_out' в словаре
            i = len(p['fc']['drop_out'])
            for _ in range(i, p['fc']['hl']):
                p['fc']['drop_out'].append(p['fc']['drop_out'][i - 1])

        if len(p['conv']['drop_out']) != p['conv']['hl']:
            i = len(p['conv']['drop_out'])
            for _ in range(i, p['conv']['hl']):
                p['conv']['drop_out'].append(p['conv']['drop_out'][i - 1])
        
        for n in ('conv', 'fc'):
            if len(p[n]['batch_norm']) != p[n]['hl']:  # дополнение раздела 'batch_norm' в словаре
                i = len(p[n]['batch_norm'])
                for _ in range(i, p[n]['hl']):
                    p[n]['batch_norm'].append(False)
            for i in range(p[n]['hl']):
                if p[n]['batch_norm'][i]:
                    p[n]['batch_norm'][i] = torch.nn.BatchNorm2d(p[n]['hn'][i]) if n == 'conv' else torch.nn.BatchNorm1d(p[n]['hn'][i])
                else:
                    p[n]['batch_norm'][i] = None
         
        self.params = p

    def convolution_step(self, x):
        """
        Обработка входной информации свёрточными слоями (input -> convolution)
        :param x (Tensor): Входные данные: матрица признаков объекта
        :return: Вектор карт признаков, полученных в результате свёртки и пулинга
        """
        for i in range(self.params['conv']['hl']):
            x = self.params['conv']['convolutions'][i](x)  # Свёртка 
            if self.params['conv']['activations'][i]:
                x = self.params['conv']['activations'][i](x)   # Пропуск через функцию активации (если указана)
            if self.params['conv']['drop_out'][i]:  # Обнуление некоторых нейронов (drop_out, если указан)
                x = self.params['conv']['drop_out'][i](x)
            if self.params['conv']['batch_norm'][i]:  # Нормализация батчей (если указана)
                x = self.params['conv']['batch_norm'][i](x)
            if self.params['conv']['pool_size'][i]:
                x = self.params['conv']['pool'][i](x)  # Пулинг получившихся карт (если указан)
            
        return x

    def fc_step(self, x):
        """
        Обработка входной информации полносвязными слоями (convolution -> fc)
        :param x (Tensor): Выходные данные для слоёв свёртки
        :return: Вероятность принадлежности к каждому классу
        """
        x = x.view(-1, self.params['fc']['input'])  # Представление входных даных в виде батчей векторов
        for i in range(self.params['fc']['hl']):
            x = self.params['fc']['linears'][i](x)  # Пропуск через линейный слой
            if self.params['fc']['activations'][i]:
                x = self.params['fc']['activations'][i](x)  # Пропуск через функцию активации (если указана)
            if self.params['fc']['drop_out'][i]:
                x = self.params['fc']['drop_out'][i](x)  # Обнуление некоторых нейронов (drop_out, если указан)
            if self.params['fc']['batch_norm'][i]:
                x = self.params['fc']['batch_norm'][i](x)  # Нормализация батчей (если указана)
        return self.params['fc']['linears'][-1](x)

    def forward(self, x):
        """
        Обработка входных данных нейронной сетью
        :param x (Tensor): Входные данные: матрица признаков объекта
        :return: Вероятность принадлежности к каждому классу
        """
        return self.fc_step(self.convolution_step(x))


In [17]:
csv_file = pd.read_csv('../input/almaz-antey-hackathon-l0/train_classification.csv', index_col=0)

In [20]:
path = '../input/almaz-antey-hackathon-l0/train/train/'
train_files_set = sorted(list(Path(path).rglob('*jpg')))
test_files = sorted(list(Path('../input/almaz-antey-hackathon-l0/test/test/').rglob('*jpg')))

In [21]:
picture_size = (768, 768)

In [22]:
trainset = ShipsDataset(files=train_files_set, mode='train', transform=transforms.Compose([
    transforms.Resize(picture_size),
    transforms.ToTensor()
]), csv_file=csv_file)

testset = ShipsDataset(files=test_files, mode='test', transform=transforms.Compose([
    transforms.Resize(picture_size),
    transforms.ToTensor()
]))

valset = None

In [23]:
testloader = DataLoader(testset, batch_size=8, shuffle=False)

Здесь назначаем все параметры для сети, тренировки, валидации и тестирования

In [24]:
model_params = dict()
model_params['conv'] = dict()
model_params['fc'] = dict()
model_params['name'] = str(next_cnn('CNNs.txt'))  # Имя нейронной сети (использовать номера)
model_params['new'] = True  # Новая сеть или нет. Если новая, будет совершена попытка загрузить данные из файла "CNN<name>.txt"
model_params['write'] = True  # Нужно ли записывать данные при обучении нейросети 

model_params['conv']['input']        = 1  # Количество каналов во входном изображении
model_params['conv']['hn']           = [64, 64, 32, 64, 64]  # Количество фильтров на каждом слое
model_params['conv']['hl']           = len(model_params['conv']['hn'])  # Количество слоёв
model_params['conv']['conv_size']    = [5, 7, 5, 5, 3]  # Размер фильтра на каждом слое
model_params['conv']['pool_size']    = [3, 2, 3, 2, 3]  # Размер ядра пулинга на каждом слое
model_params['conv']['conv_stride']  = [2, 1, 1, 1, 1]  # Величина шага фильтра свёртки на каждом слое (stride)
model_params['conv']['pool_stride']  = [2, 2, 2, 2, 2]  # Величина шага ядра пулинга на каждом слое (stride)
model_params['conv']['batch_norm']   = [True, False, True, False, True]  # Нормализация батча для каждого слоя 
model_params['conv']['convolutions'] = conv_addition(model_params['conv']['input'], model_params['conv']['hn'], model_params['conv']['hl'],
                                                     model_params['conv']['conv_size'], model_params['conv']['conv_stride'])
model_params['conv']['pool']         = pool_addition(model_params['conv']['hl'], model_params['conv']['pool_size'], 
                                                     model_params['conv']['pool_stride'])
model_params['conv']['activations']  = [torch.nn.ReLU(), torch.nn.ReLU(), torch.nn.ReLU(), torch.nn.ReLU(), torch.nn.ReLU()]
model_params['conv']['drop_out']     = [None, torch.nn.Dropout(), None, torch.nn.Dropout(), None]  # Указание Dropout для каждого слоя (автоматически дополняется)


h, w = picture_size # Указание высоты и ширины входной картинки
h, w = maps_dims([h, w], model_params['conv']['conv_size'], model_params['conv']['pool_size'], 
                 model_params['conv']['conv_stride'], model_params['conv']['pool_stride'])
print('output map height: {};\toutput map width: {};\n'.format(h, w))
print('model name: {}'.format(model_params['name']))


model_params['fc']['input']       = h * w * model_params['conv']['hn'][-1]  # Количество входных данных (количество input-нейронов)
model_params['fc']['output']      = 2  # Количество выходных данных (Количество output-нейронов)
model_params['fc']['hn']          = [1000]# Количество нейронов на каждом слое
model_params['fc']['hl']          = len(model_params['fc']['hn'])  # Количество полносвязных слоёв
model_params['fc']['activations'] = [torch.nn.ReLU()]  # Объявление функций активации для каждого слоя
model_params['fc']['linears']     = linears_addition(model_params['fc']['input'], model_params['fc']['hn'], 
                                                 model_params['fc']['hl'], model_params['fc']['output'])  # Объявление линейных слоёв
model_params['fc']['drop_out']    = [torch.nn.Dropout(), torch.nn.Dropout()]  # Указание Dropout для каждого слоя (автоматически дополняется)
model_params['fc']['batch_norm']  = [True, True]  # Указание BatchNormalization для каждого слоя (дополняется автоматически)


output map height: 8;	output map width: 8;

model name: 1


In [25]:
train_params                   = dict() 
train_params['lr']             = 0.0005  # первоначальный learning_rate
train_params['lr_decrease']    = 2  # Уменьшение learning_rate после каждой эпохи. Если нет необходимости, выставить 1
train_params['num_epochs']     = 4  # Количество эпох
train_params['write_seq']      = 50  # Частота записи лоссов обучаемой нейросети
train_params['show_seq']       = 20  # Частота отображения лоссов обучаемой нейросети
train_params['loss_fn']        = torch.nn.CrossEntropyLoss()  # Функция потерь
train_params['trainset']    = trainset  # Датасет для тренировки
train_params['testset']     = testset
train_params['primary_net']    = None  # Изначальная сеть
train_params['final_net']      = None  # Обученная сеть
train_params['losses_path']    = 'Losses.txt'  # Путь для сохранения лоссов (в kaggle kernels приходится дописывать полный путь в функциях)
train_params['networks_path']  = 'CNNs.txt'  # Путь для сохранения данных о нейросети
train_params['primary_batch']  = 64  # Изначальный размер батчей
train_params['batch_decrease'] = 2  # Уменьшение размера батчей каждые две эпохи. Если нет необходимости, выставить 1


In [38]:
val_params = dict()
val_params['batch_size'] = 16  # Размер батчей
val_params['valset'] = valset  # Датасет для валидации
val_params['loss_fn'] = torch.nn.functional.cross_entropy  # Функция потерь (При использовании класса в torch.nn по необъяснимым причинам
# 16 GB RAM уничтожаются за несколько секунд, поэтому используется данный вариант)
val_params['net'] = trainset  # Нейросеть для валидации

In [27]:
test_params = dict()
test_params['testset'] = testset  # Датасет для тестирования
test_params['batch_size'] = 16  # Размер батчей
test_params['net'] = None  # Нейросеть для тестирования

Так как мы записывали все слои в виде словаря и в классе нейросети они будут храниться в виде списков, из которых просто будет извлекаться информация о конкретном слое, то параметры сети остаются пустыми. Это приводит к тому, что torch.nn.module просто не понимает, что мы от него хотим и выдаёт ошибку или не обучается (потому что обучаться нечему, как он считает).

Поэтому все параметры придётся назначать вручную. Написав данный код один раз, про него можно забыть

In [28]:
if not model_params['new']: 
    try:
        train_params['primary_net'] = torch.load('../input/kernel0fc93dd4ed/CNN{}'.format(model_params['name']))
    except:
        model_params['new'] = True
if model_params['new']:
    train_params['primary_net'] = ConvNetwork(model_params)  # Инициализация сети

    for i in range(model_params['conv']['hl']):  # Инициализация параметров свёрточных слоёв сети
        train_params['primary_net'].__setattr__('convolution_{}'.format(i + 1), model_params['conv']['convolutions'][i])
        if model_params['conv']['activations'][i]:
            train_params['primary_net'].__setattr__('conv_activation_{}'.format(i + 1), model_params['conv']['activations'][i])
        if model_params['conv']['batch_norm'][i]:
            train_params['primary_net'].__setattr__('conv_batch_norm_{}'.format(i + 1), model_params['conv']['batch_norm'][i])
        if model_params['conv']['pool_size'][i]:
            train_params['primary_net'].__setattr__('pool_{}'.format(i + 1), model_params['conv']['pool'][i])

    for i in range(model_params['fc']['hl']):  # Инициализация параметров полносвязных слоёв сети
        train_params['primary_net'].__setattr__('linear_{}'.format(i + 1), model_params['fc']['linears'][i])
        if model_params['fc']['activations'][i]:
            train_params['primary_net'].__setattr__('fc_activation_{}'.format(i + 1), model_params['fc']['activations'][i])
        if model_params['fc']['drop_out'][i]:
            train_params['primary_net'].__setattr__('drop_{}'.format(i + 1), model_params['fc']['drop_out'][i])
        if model_params['fc']['batch_norm'][i]:
            train_params['primary_net'].__setattr__('fc_batch_norm_{}'.format(i + 1), model_params['fc']['batch_norm'][i])
    train_params['primary_net'].__setattr__('linear_{}'.format(model_params['fc']['hl'] + 1), model_params['fc']['linears'][-1])



In [29]:
train_params['primary_net'].cuda()
print(train_params['primary_net'])

ConvNetwork(
  (convolution_1): Conv2d(1, 64, kernel_size=(5, 5), stride=(2, 2))
  (conv_activation_1): ReLU()
  (conv_batch_norm_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool_1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (convolution_2): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1))
  (conv_activation_2): ReLU()
  (pool_2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (convolution_3): Conv2d(64, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv_activation_3): ReLU()
  (conv_batch_norm_3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool_3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (convolution_4): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv_activation_4): ReLU()
  (pool_4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (convolution_5): Conv2d(64, 64, kern

In [30]:
"Обучение нейросети"
train_params['final_net'] = train(train_params) if model_params['new'] else train_params['primary_net']
val_params['net'], test_params['net'] = train_params['final_net'], train_params['final_net']


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:34: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


[1,    20] loss: 0.637
[1,    40] loss: 0.615
[1,    60] loss: 0.609
[1,    80] loss: 0.499
[1,   100] loss: 0.485
[1,   120] loss: 0.487
[1,   140] loss: 0.494
[1,   160] loss: 0.472
[1,   180] loss: 0.456
[1,   200] loss: 0.432



[2,    20] loss: 0.443
[2,    40] loss: 0.398
[2,    60] loss: 0.395
[2,    80] loss: 0.341
[2,   100] loss: 0.363
[2,   120] loss: 0.386
[2,   140] loss: 0.415
[2,   160] loss: 0.329
[2,   180] loss: 0.336
[2,   200] loss: 0.383



[3,    20] loss: 0.348
[3,    40] loss: 0.313
[3,    60] loss: 0.382
[3,    80] loss: 0.340
[3,   100] loss: 0.373
[3,   120] loss: 0.362
[3,   140] loss: 0.350
[3,   160] loss: 0.347
[3,   180] loss: 0.366
[3,   200] loss: 0.351
[3,   220] loss: 0.320
[3,   240] loss: 0.328
[3,   260] loss: 0.317
[3,   280] loss: 0.281
[3,   300] loss: 0.312
[3,   320] loss: 0.308
[3,   340] loss: 0.329
[3,   360] loss: 0.339
[3,   380] loss: 0.295
[3,   400] loss: 0.293



[4,    20] loss: 0.304
[4,    40] loss: 0.354
[4,    60] loss: 0.335
[4,    80] loss: 0.315
[4,   100] loss: 0.299
[4,   120] loss: 0.320
[4,   140] loss: 0.293
[4,   160] loss: 0.319
[4,   180] loss: 0.327
[4,   200] loss: 0.312
[4,   220] loss: 0.288
[4,   240] loss: 0.286
[4,   260] loss: 0.311
[4,   280] loss: 0.262
[4,   300] loss: 0.276
[4,   320] loss: 0.305
[4,   340] loss: 0.259
[4,   360] loss: 0.286
[4,   380] loss: 0.283
[4,   400] loss: 0.292


Обучение закончено


In [32]:
test(test_params)

Тестирование завершено, результаты сохранены
